In [1]:
import sys
import os
sys.path.append("..")
import numpy as np
from utils import create_metadata_df, measurements_to_df
import json
from send2trash import send2trash
import shutil

/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Delete unused sensor folders

In [ ]:
def delete_sensor_folders(root_dir, inputs_to_be_deleted):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type in inputs_to_be_deleted:
                shutil.rmtree(root)

In [ ]:
root_dir = os.path.join("..", "..", "data", "data")
inputs_to_be_deleted = ["depth", "label_raw", "semantics", "topdown"]
delete_sensor_folders(root_dir, inputs_to_be_deleted)

## Delete NaN values files

In [3]:
path_data = os.path.join("..", "..", "data")
df_meta_data = create_metadata_df(path_data, ["rgb", "lidar", "measurements"])

In [4]:
df_measurements = measurements_to_df(df_meta_data)

100%|██████████| 270681/270681 [01:33<00:00, 2909.55it/s]


In [14]:
idx_nan_list = df_measurements[df_measurements.isna().any(axis=1)].index.values

In [ ]:
def remove_entries_at_indices(idx_nan_list, df_meta_entire):
    count_files_deleted = 0
    for idx_nan in idx_nan_list:
        path_nan = os.path.join(df_meta_entire.loc[idx_nan]["dir"], "measurements", df_meta_entire.loc[idx_nan]["measurements"])
        with open(path_nan, 'r') as f:
            measurements = json.load(f)
        if not np.isnan(measurements["steer"]):
            print("Break: indexes do not fit!")
            break
        dir = df_meta_entire["dir"].loc[idx_nan]
        dirs_sensors = os.listdir(dir)
        number_entry = df_meta_entire["measurements"].loc[idx_nan].split(".")[0]
        for dir_sensor in dirs_sensors:
            if not dir_sensor.startswith("."):
                files = os.listdir(os.path.join(dir, dir_sensor))
                for file in files:
                    if file.startswith(number_entry):
                        count_files_deleted += 1
                        # os.remove(os.path.join(dir, dir_sensor, file))
                        send2trash(os.path.join(dir, dir_sensor, file))
    return count_files_deleted

In [20]:
remove_entries_at_indices(idx_nan_list, df_meta_data)

## Remove invalid files from Noise-Dataset

In [ ]:
def remove_all_not_rgb_existent_files(root_dir):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type == "rgb":
                file_numbers_rgb = [filename.split(".")[0] for filename in files]
                # Delete measurements files not in rgb
                file_numbers_measurements = [filename.split(".")[0] for filename in os.listdir(os.path.join(dir, "measurements"))]
                for file_number in file_numbers_measurements:
                    if file_number not in file_numbers_rgb:
                        os.remove(os.path.join(dir,"measurements", f"{file_number}.json"))
                # Delete lidar files not in rgb
                file_numbers_lidar = [filename.split(".")[0] for filename in os.listdir(os.path.join(dir, "lidar"))]
                for file_number in file_numbers_lidar:
                    if file_number not in file_numbers_rgb:
                        os.remove(os.path.join(dir,"lidar", f"{file_number}.npy"))   

In [ ]:
path_noise_data = os.path.join("..", "..", "data", "noise_data")
if os.path.exists(path_noise_data):
    remove_all_not_rgb_existent_files(path_noise_data)